<a href="https://colab.research.google.com/github/arnavdodiedo/RL-Algorithms/blob/main/GridWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

## Generalized policy iteration with DP, Monte Carlo, TD learning and Q learning
#### where the initial policy is a uniform one, equal probabilites of going to each neighbouring state

In [55]:
class gridworld():
    def __init__(self, size = 4, reward = 0, penalty_per_step = -1, epochs = 100, alpha = 0.9, gamma = 0.99, epsilon_decay_factor = 0.99, seed = 0):
        self.size = size # square grid world size
        self.epsilon_decay_factor = epsilon_decay_factor
        self.returngrid = np.zeros((size, size), dtype=np.float) # reward for all states initialised to 0   (state values for each state)     
        self.goal_states = [[0,0],[size-1, size-1]] # goal states are the top left and bottom right corners of the grid
        self.reward = reward # 0 reward on reaching goal state
        self.epsilon = 1
        np.random.seed(seed)

        # terminate on reaching goal state OR stuck in infinite loop with 0 rewards per step in this loop

        self.policy = np.zeros((self.size, self.size, 4)) + 0.25 # set initial policy as naive, equal probabilities in all directions
        self.policy[self.goal_states[0][0],self.goal_states[0][1]] = \
                self.policy[self.goal_states[1][0], self.goal_states[1][1]] = np.zeros(4) # set movement probability for goal state as all 0

        self.state_action_values = np.zeros((self.size, self.size, 4)) # state action value matrix

        self.penalty_per_step = penalty_per_step # -1 reward per movement 
        self.movements = [[-1,0], [1,0], [0,-1], [0,1]]
        self.episodes = self.epochs = epochs # number of epochs to run policy evalution
        self.gamma = gamma
        self.alpha = alpha

    def reset_grid(self): # reset all agent and environment variables
        self.returngrid = np.zeros((self.size, self.size))
        self.epsilon = 1
        self.policy = np.zeros((self.size, self.size, 4)) + 0.25 # set initial policy as naive, equal probabilities in all directions
        self.policy[self.goal_states[0][0],self.goal_states[0][1]] = \
                self.policy[self.goal_states[1][0], self.goal_states[1][1]] = np.zeros(4) # set movement probability for goal state as all 0
        self.state_action_values = np.zeros((self.size, self.size, 4)) # reset state action value matrix

    def greedy_policy_update_dp(self, curr_state):  # update policy greedily, second step of generalized policy iteration (GPI)
        if curr_state not in self.goal_states:
            self.policy[curr_state[0]][curr_state[1]] = np.zeros(4)
            
            up = self.next_state(curr_state, 0)
            up = self.returngrid[up[0]][up[1]]

            down = self.next_state(curr_state, 1)
            down = self.returngrid[down[0]][down[1]]
            
            left = self.next_state(curr_state, 2)
            left = self.returngrid[left[0]][left[1]]
            
            right = self.next_state(curr_state, 3)
            right = self.returngrid[right[0]][right[1]]

            arr = [[up, 0], [down, 1], [left, 2], [right, 3]]            
            arr.sort()       
            arr.reverse()     

            m = [arr[0][1]]
            for i in range(1, 4):
                if arr[i][0] != arr[i-1][0]: break
                else: m.append(arr[i][1])
            
            value = 1/len(m)
            
            for i in m:
                self.policy[curr_state[0]][curr_state[1]][i] = value
            
            # print(self.policy[curr_state[0]][curr_state[1]])


    def next_state(self, curr_state, direction): # direction = 0 - up, 1 - down, 2 - left, 3 - right
        change = self.movements[direction]
        new_state = [curr_state[0]+change[0], curr_state[1]+change[1]]        

        if(new_state[0]<0): new_state[0] = 0
        elif(new_state[0]>self.size-1): new_state[0] = self.size-1
        
        if(new_state[1]<0): new_state[1] = 0
        elif(new_state[1]>self.size-1): new_state[1] = self.size-1

        return new_state

    def display_grid(self): # display the state of the grid world                
        print("[", end="")
        for i in range(self.size):
            print("\n       [", end=" ")
            for j in range(self.size):
                print(self.returngrid[i][j], end=", ")
            print("]", end="")
        print("\n]")

    def display_state_action_values(self): # display state action values
        print("[", end="")
        for i in range(self.size):
            print("\n       [", end=" ")
            for j in range(self.size):
                print(self.state_action_values[i][j], end=", ")
            print("]", end="")
        print("\n]")

    def display_policy(self): # display policy at each state, for each of the 4 actions
        print("[", end="")
        for i in range(self.size):
            print("\n       [", end=" ")
            for j in range(self.size):                
                
                if [i,j] in self.goal_states: 
                    print("stay", end=", ")
                    continue

                probs = [[self.policy[i][j][0], 0], [self.policy[i][j][1], 1], [self.policy[i][j][2], 2], [self.policy[i][j][3], 3]]
                probs.sort()
                probs.reverse()

                moves = ["up", "down", "left", "right"]
                m = [probs[0][1]]

                for k in range(1,4):
                    if probs[k][0] != probs[k-1][0]: break
                    else: m.append(probs[k][1])
                
                for k in range(len(m)):
                    if k!=len(m)-1: print(moves[m[k]], end="&")
                    else: print(moves[m[k]], end=", ")
                
            print("]", end="")
        print("\n]")

    def evaluate_current_policy_dp(self): # policy evaluation using dp
        self.reset_grid() # start from scratch
        for _ in range(1, 1+self.epochs):                        
            for i in range(self.size):
                for j in range(self.size):                    
                    value = 0
                    for k in range(4):
                        new_state = self.next_state([i,j], k)
                        value += (self.penalty_per_step + self.gamma * self.returngrid[new_state[0], new_state[1]]) * self.policy[i][j][k]                                
                    
                    self.returngrid[i][j] = value

            if (_%10==0):
                print("epoch #%d"%(_), end=" ")
                self.display_grid()                
    
    def gpi_dp(self): # GPI using dp
        self.reset_grid() # start from scratch
        for _ in range(1, 1+self.epochs):
            # either update in self.returngrid as you traverse OR in each epoch maintain a copy of self.returngrid and use it for update at the end
            returngrid_copy = np.copy(self.returngrid)
            for i in range(self.size):
                for j in range(self.size):                    
                    value = 0
                    for k in range(4):
                        new_state = self.next_state([i,j], k)
                        value += (self.penalty_per_step+self.gamma * self.returngrid[new_state[0]][new_state[1]]) * self.policy[i][j][k]                    
                    returngrid_copy[i,j] = value

            if (self.returngrid == returngrid_copy).all():                 
                print("Policy did not improve. Stopping...")
                print("epoch #%d"%(_), end=" ")
                self.display_grid()
                print("policy -")
                self.display_policy()
                break
            else: 
                self.returngrid = returngrid_copy
            
            for i in range(self.size): 
                for j in range(self.size): 
                    self.greedy_policy_update_dp([i,j])            

            if (_%10==0):
                print("epoch #%d"%(_), end=" ")
                self.display_grid()
                print("policy -")
                self.display_policy()

    def evaluate_current_policy_monte_carlo(self): # evaluate policy using first visit monte carlo
        self.reset_grid() # start from scratch

        number_of_visits = np.zeros((self.size, self.size))
        for _ in range(1, 1+self.episodes):
            is_visited_this_episode = np.zeros((self.size, self.size))

            k = np.random.choice(range(1, self.size*self.size-1))
            i = k//4
            j = k - 4*i                        
            
            state = [i, j]
            state_reward = []                        
            path = []

            if state in self.goal_states: continue

            while state not in self.goal_states:
                if is_visited_this_episode[state[0]][state[1]] == 0:
                    number_of_visits[state[0],state[1]] += 1
                    is_visited_this_episode[state[0]][state[1]] = 1
                
                path.append(state)
                state_reward.append(self.penalty_per_step)
                direction = np.random.choice(range(len(self.movements)), p=self.policy[state[0]][state[1]])
                new_state = self.next_state(state, direction)            
                state = new_state
                
            state_returns = np.zeros_like(state_reward, dtype=np.float)                        
            state_returns[-1] = state_reward[-1]

            for i in range(len(state_reward)-2, -1, -1):
                state_returns[i] = state_reward[i] + self.gamma * state_returns[i+1]
            
            is_visited_this_episode = np.zeros((self.size, self.size))

            for p in range(len(path)):
                if is_visited_this_episode[path[p][0]][path[p][1]] == 1: continue
                
                is_visited_this_episode[path[p][0]][path[p][1]] = 1
                
                self.returngrid[path[p][0]][path[p][1]] += (state_returns[p]-self.returngrid[path[p][0]][path[p][1]])/number_of_visits[path[p][0]][path[p][1]]
            
            if (_%(self.episodes/10) == 0):
                print("epoch #%d"%(_), end=" ")
                self.display_grid()
                # print("policy - ")
                # self.display_policy()
    
    def epsilon_greedy_policy_update_monte_carlo(self): # epsilon greedy update of policy, second step of monte carlo policy improvement
        
        for i in range(self.size):
            for j in range(self.size):

                if [i,j] in self.goal_states: continue

                prob = np.zeros(len(self.movements)) + self.epsilon/len(self.movements)        
                prob[np.argmax(self.state_action_values[i][j])] = 1 - self.epsilon + self.epsilon/len(self.movements)

                self.policy[i][j] = prob
            
                # print("updated policy at", curr_state, "to", self.policy[i][j])

    def gpi_monte_carlo(self): # GPI using epsilon greedy first visit monte carlo
        self.reset_grid()
        number_of_visits = np.zeros((self.size, self.size, 4))
        for _ in range(1, 1+self.episodes):
            # print("epoch", _)
            is_visited_this_episode = np.zeros((self.size, self.size, 4))
            
            k = np.random.choice(range(1, self.size*self.size-1))            
            i = k//4
            j = k - 4*i                                    
            
            action = np.random.choice([0,1,2,3])
            
            state = [i, j]
            state_reward = []                        
            path = []            

            while True:
                if is_visited_this_episode[state[0]][state[1]][action] == 0:
                    number_of_visits[state[0]][state[1]][action] += 1
                    is_visited_this_episode[state[0]][state[1]][action] = 1

                # print("in state", state, "took action", action, "with policy", self.policy[state[0]][state[1]])                             

                path.append([state, action])
                state_reward.append(self.penalty_per_step)

                # IMPORTANT STEP - first select next state based on current state and action, 
                # then select next action based on probabilities in this new state
                state = self.next_state(state, action)                                

                if state in self.goal_states: break
        
                action = np.random.choice(range(len(self.movements)), p=self.policy[state[0]][state[1]])
                
            state_action_returns = np.zeros_like(state_reward, dtype=np.float)                        
            state_action_returns[-1] = state_reward[-1]

            for i in range(len(state_reward)-2, -1, -1):
                state_action_returns[i] = state_reward[i] + self.gamma * state_action_returns[i+1]
                # print(state_returns[i], self.gamma * state_returns[i+1], end=",")
            # print("\n")

            is_visited_this_episode = np.zeros((self.size, self.size, 4))
            for p in range(len(path)):
                x, y, a = path[p][0][0], path[p][0][1], path[p][1]

                if is_visited_this_episode[x][y][a] == 1: continue
                
                is_visited_this_episode[x][y][a] = 1   

                # self.state_action_values[x][y][a] += (state_action_returns[p]-self.state_action_values[x][y][a])/number_of_visits[x][y][a]
                self.state_action_values[x][y][a] = np.mean(state_action_returns[p:])
                        
            self.epsilon_greedy_policy_update_monte_carlo() # improve the policy based on updated state-action values                                                        

            if (_%(self.epochs/10) == 0):                
                print("epoch #%d"%(_), end=" ")
                self.display_state_action_values()
                print("policy - ")
                self.display_policy()  

                self.epsilon *= self.epsilon_decay_factor   

    def evaluate_current_policy_tdlearning(self):
        self.reset_grid()
        for _ in range(1, 1+self.episodes):
            k = np.random.choice(range(1, self.size*self.size-1))

            i = k//4
            j = k - 4*i
            state = [i,j]            

            while state not in self.goal_states:
                action = np.random.choice([0,1,2,3], p=self.policy[i][j])            
                new_state = self.next_state(state, action)
                self.returngrid[state[0]][state[1]] += self.alpha * (self.penalty_per_step + self.gamma * self.returngrid[new_state[0]][new_state[1]] - self.returngrid[state[0]][state[1]])
                state = new_state

            if _%(self.episodes/10)==0:
                print("epoch #%d"%(_), end=" ")
                self.display_grid()                

## Dynamic Programming

In [60]:
grid = gridworld(epochs=100, gamma = 0.9, seed = 0)
# grid.evaluate_current_policy_dp()
grid.gpi_dp()

Policy did not improve. Stopping...
epoch #4 [
       [ 0.0, -1.0, -2.0, -3.0, ]
       [ -1.0, -2.0, -3.0, -2.0, ]
       [ -2.0, -3.0, -2.0, -1.0, ]
       [ -3.0, -2.0, -1.0, 0.0, ]
]
policy -
[
       [ stay, left, left, left&down, ]
       [ up, left&up, right&left&down&up, down, ]
       [ up, right&left&down&up, right&down, down, ]
       [ right&up, right, right, stay, ]
]


## Monte Carlo 

In [44]:
grid = gridworld(size = 4, epochs=100, gamma = 0.8, epsilon_decay_factor = 0.5, seed = 0)
# grid.evaluate_current_policy_monte_carlo()
grid.gpi_monte_carlo()

epoch #10 [
       [ [0. 0. 0. 0.], [-1.4        -4.45959979 -1.         -3.0760394 ], [-4.48726469 -4.67741967 -1.4        -4.72222225], [-2.54048    -2.31072    -4.68253993 -4.69230785], ]
       [ [-1.         -2.54048    -1.74666667 -4.74025975], [-1.74666667 -3.0760394  -2.74204343 -4.73684212], [-1.74666667 -4.73333335 -3.21474836 -3.44786579], [-4.69696982 -4.71428576 -3.33799881 -3.21474836], ]
       [ [-2.048      -4.05640321 -4.74683545 -2.31072   ], [-2.9194304  -3.0760394  -4.75       -3.63425781], [-3.54611628 -4.66666718 -4.62264427 -4.64285848], [-4.71014499 -1.         -4.62963179 -4.63636534], ]
       [ [-2.31072    -4.01152922 -2.54048    -3.96253844], [-3.90890489 -2.54048    -2.74204343 -2.9194304 ], [-4.6610176  -2.048      -2.74204343 -1.        ], [0. 0. 0. 0.], ]
]
policy - 
[
       [ stay, right&left&down&up, right&left&down&up, right&left&down&up, ]
       [ right&left&down&up, right&left&down&up, right&left&down&up, right&left&down&up, ]
       [ right&lef

##TD Learning

In [61]:
grid = gridworld(size=4, epochs=100, alpha = 0.4, gamma=0.9, seed=0)
grid.evaluate_current_policy_tdlearning()

epoch #10 [
       [ 0.0, -0.4, 0.0, 0.0, ]
       [ -2.9629724360937026, -2.639936981081504, 0.0, 0.0, ]
       [ -5.44151317331102, -4.014618934682042, -1.7166933635267365, -0.4, ]
       [ -6.166773633673814, -4.727311671708557, -2.3115101357220333, 0.0, ]
]
epoch #20 [
       [ 0.0, -3.6906582799099343, -6.292470090417065, -6.500122623042976, ]
       [ -3.5077005430554573, -6.31259152772494, -6.566089336250388, -6.33824278069204, ]
       [ -5.998135981012131, -6.581771664356883, -4.915559162026243, -1.9986345158012115, ]
       [ -7.317762953663052, -6.085004646856289, -5.213237152387899, 0.0, ]
]
epoch #30 [
       [ 0.0, -5.880178946660615, -7.35219747458223, -7.5045626184122245, ]
       [ -2.6167744394189896, -6.340465641471173, -7.130925611995167, -5.888951729411248, ]
       [ -7.250678064283298, -7.189584699331956, -6.797452462621809, -3.614429250769203, ]
       [ -7.782966144868626, -6.818831956791831, -3.3627339164741805, 0.0, ]
]
epoch #40 [
       [ 0.0, -4.8441819016

## Tester cells (to be ignored)

In [26]:
np.arange(1,15)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])